In [1]:
import os
import numpy as np
import dask.array as da
from dask.distributed import Client
from functions import kernel_est_funcs as kef
from functions import conn_inf_funcs as cif
from functions import remove_outliers as ro
import h5py

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
file_path = '/home/joudy/Documents/projects/data/spikes-2.h5'
gt_path = '/home/joudy/Documents/projects/data/connectivity-h5-2.npy'

In [3]:
G = np.load(gt_path)

In [4]:
spikes_hdf = h5py.File(file_path, 'r')
print(list(spikes_hdf.keys()))
spikes_train = spikes_hdf['spikes-ds']  
spikes_train.shape

['spikes-ds']


(1250, 1000000)

In [7]:
client = Client()

dashboard_link = client.dashboard_link
print(f"Dask Dashboard: {dashboard_link}")
import webbrowser
# Optionally, open it automatically in the default browser
webbrowser.open(dashboard_link)

dask_array = da.from_array(spikes_train, chunks=('auto', 1000000))
pr1 = dask_array.map_blocks(kef.dask_calcium, dtype=dask_array.dtype)
pr2 = pr1.map_blocks(kef.dask_smooth, dtype=dask_array.dtype)
#pr3 = pr2.map_blocks(cif.dask_conn_inf_LR, gt_path, dtype=dask_array.dtype)
result = pr2.compute()


# Check the result
print("Processing completed!")

/home/joudy/miniconda3/envs/phd_conda/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 45213 instead
  warnings.warn(


Dask Dashboard: http://127.0.0.1:45213/status
Processing completed!


In [9]:
with h5py.File('feed-signal.h5', 'w') as hdf:
    hdf.create_dataset('signals', data=result)

In [ ]:
cor_result = cif.conn_inf_LR(result, gt_path)
print(cor_result)

!!! this should be test for a one-time run pipeline

In [6]:
client = Client()

dashboard_link = client.dashboard_link
print(f"Dask Dashboard: {dashboard_link}")
import webbrowser
# Optionally, open it automatically in the default browser
webbrowser.open(dashboard_link)

dask_array = da.from_array(spikes_train, chunks=('auto', 1000000))
pr1 = dask_array.map_blocks(kef.dask_calcium, dtype=dask_array.dtype)
pr2 = pr1.map_blocks(kef.dask_smooth, dtype=dask_array.dtype)
pr3 = pr2.map_blocks(cif.dask_conn_inf_LR, gt_path, dtype=dask_array.dtype)
result = pr3.compute()


# Check the result
print("Processing completed!")

/home/joudy/miniconda3/envs/phd_conda/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41313 instead
  warnings.warn(


Dask Dashboard: http://127.0.0.1:41313/status


2024-10-21 12:43:37,279 - distributed.worker - ERROR - Compute Failed
Key:       ('dask_conn_inf_LR-54fe653b6dc9bc1100e8afcc11480969', 78, 0)
State:     executing
Function:  subgraph_callable-3621a2a46f3076373069776ab9a15a5d
args:      ('/home/joudy/Documents/projects/data/connectivity-h5-2.npy', <HDF5 dataset "spikes-ds": shape (1250, 1000000), type "<f8">, (slice(1248, 1250, None), slice(0, 1000000, None)))
kwargs:    {}
Exception: "ValueError('all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 6250000 and the array at index 1 has size 4')"
Traceback: '  File "/home/joudy/miniconda3/envs/phd_conda/lib/python3.12/site-packages/dask/optimization.py", line 1028, in __call__\n    return core.get(self.dsk, self.outkey, dict(zip(self.inkeys, args)))\n           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n  File "/home/joudy/miniconda3/envs/phd_conda/lib/python3.12/site-packages/dask/co

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 6250000 and the array at index 1 has size 4